<a href="https://colab.research.google.com/github/Tinddd28/labs-iadnmo-/blob/main/lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №4 "Классификация изображений. Сверточные сети. Полносвязные сети с предобучением и дообучением"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf